In [1]:
from tensorflow import keras 
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

In [13]:
# create a YOLOv3 Keras model and save it to file
# based on https://github.com/experiencor/keras-yolo3
import struct
import numpy as np
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.layers import UpSampling2D
from keras.layers.merge import add, concatenate
from keras.models import Model

def _conv_block(inp, convs, skip=True):
	x = inp
	count = 0
	for conv in convs:
		if count == (len(convs) - 2) and skip:
			skip_connection = x
		count += 1
		if conv['stride'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
		x = Conv2D(conv['filter'],
				   conv['kernel'],
				   strides=conv['stride'],
				   padding='valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefer left and top
				   name='conv_' + str(conv['layer_idx']),
				   use_bias=False if conv['bnorm'] else True)(x)
		if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
		if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)
	return add([skip_connection, x]) if skip else x

def make_yolov3_model():
	input_image = Input(shape=(None, None, 3))
	# Layer  0 => 4
	x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
								  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
								  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
								  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])
	# Layer  5 => 8
	x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
						{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
						{'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])
	# Layer  9 => 11
	x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
						{'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])
	# Layer 12 => 15
	x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
						{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
						{'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])
	# Layer 16 => 36
	for i in range(7):
		x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
							{'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
	skip_36 = x
	# Layer 37 => 40
	x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])
	# Layer 41 => 61
	for i in range(7):
		x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
							{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
	skip_61 = x
	# Layer 62 => 65
	x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])
	# Layer 66 => 74
	for i in range(3):
		x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
							{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
	# Layer 75 => 79
	x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)
	# Layer 80 => 82
	yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
							  {'filter':  255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)
	# Layer 83 => 86
	x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
	x = UpSampling2D(2)(x)
	x = concatenate([x, skip_61])
	# Layer 87 => 91
	x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)
	# Layer 92 => 94
	yolo_94 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
							  {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)
	# Layer 95 => 98
	x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
	x = UpSampling2D(2)(x)
	x = concatenate([x, skip_36])
	# Layer 99 => 106
	yolo_106 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
							   {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
							   {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
							   {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)
	model = Model(input_image,[yolo_82, yolo_94, yolo_106])
	#model2 = Model()
	return model

class WeightReader:
	def __init__(self, weight_file):
		with open(weight_file, 'rb') as w_f:
			major,	= struct.unpack('i', w_f.read(4))
			minor,	= struct.unpack('i', w_f.read(4))
			revision, = struct.unpack('i', w_f.read(4))
			if (major*10 + minor) >= 2 and major < 1000 and minor < 1000:
				w_f.read(8)
			else:
				w_f.read(4)
			transpose = (major > 1000) or (minor > 1000)
			binary = w_f.read()
		self.offset = 0
		self.all_weights = np.frombuffer(binary, dtype='float32')

	def read_bytes(self, size):
		self.offset = self.offset + size
		return self.all_weights[self.offset-size:self.offset]

	def load_weights(self, model):
		for i in range(106):
			try:
				conv_layer = model.get_layer('conv_' + str(i))
				print("loading weights of convolution #" + str(i))
				if i not in [81, 93, 105]:
					norm_layer = model.get_layer('bnorm_' + str(i))
					size = np.prod(norm_layer.get_weights()[0].shape)
					beta  = self.read_bytes(size) # bias
					gamma = self.read_bytes(size) # scale
					mean  = self.read_bytes(size) # mean
					var   = self.read_bytes(size) # variance
					weights = norm_layer.set_weights([gamma, beta, mean, var])
				if len(conv_layer.get_weights()) > 1:
					bias   = self.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
					kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
					kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
					kernel = kernel.transpose([2,3,1,0])
					conv_layer.set_weights([kernel, bias])
				else:
					kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
					kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
					kernel = kernel.transpose([2,3,1,0])
					conv_layer.set_weights([kernel])
			except ValueError:
				print("no convolution #" + str(i))

	def reset(self):
		self.offset = 0

# define the model
model = make_yolov3_model()
# load the model weights
weight_reader = WeightReader('yolov3.weights')
# set the model weights into the model
weight_reader.load_weights(model)
# save the model to file
model.save('model_v3.h5')

loading weights of convolution #0
loading weights of convolution #1
loading weights of convolution #2
loading weights of convolution #3
no convolution #4
loading weights of convolution #5
loading weights of convolution #6
loading weights of convolution #7
no convolution #8
loading weights of convolution #9
loading weights of convolution #10
no convolution #11
loading weights of convolution #12
loading weights of convolution #13
loading weights of convolution #14
no convolution #15
loading weights of convolution #16
loading weights of convolution #17
no convolution #18
loading weights of convolution #19
loading weights of convolution #20
no convolution #21
loading weights of convolution #22
loading weights of convolution #23
no convolution #24
loading weights of convolution #25
loading weights of convolution #26
no convolution #27
loading weights of convolution #28
loading weights of convolution #29
no convolution #30
loading weights of convolution #31
loading weights of convolution #32

In [46]:
#model_yolo_94 = tf.keras.models.load_model('model_v3.h5')
#model_yolo_82 = tf.keras.models.load_model('model_v3.h5')
#model_yolo_106 = tf.keras.models.load_model('model_v3.h5')
model = tf.keras.models.load_model('model_v3.h5')

In [180]:
model = tf.keras.models.load_model('model_v3.h5')
model.compile()

In [146]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv_0 (Conv2D)                (None, None, None,   864         ['input_9[0][0]']                
                                32)                                                               
                                                                                                  
 bnorm_0 (BatchNormalization)   (None, None, None,   128         ['conv_0[0][0]']                 
                                32)                                                         

                                                                                                  
 bnorm_9 (BatchNormalization)   (None, None, None,   256         ['conv_9[0][0]']                 
                                64)                                                               
                                                                                                  
 leaky_9 (LeakyReLU)            (None, None, None,   0           ['bnorm_9[0][0]']                
                                64)                                                               
                                                                                                  
 conv_10 (Conv2D)               (None, None, None,   73728       ['leaky_9[0][0]']                
                                128)                                                              
                                                                                                  
 bnorm_10 

                                256)                                                              
                                                                                                  
 bnorm_20 (BatchNormalization)  (None, None, None,   1024        ['conv_20[0][0]']                
                                256)                                                              
                                                                                                  
 leaky_20 (LeakyReLU)           (None, None, None,   0           ['bnorm_20[0][0]']               
                                256)                                                              
                                                                                                  
 add_5 (Add)                    (None, None, None,   0           ['add_4[0][0]',                  
                                256)                              'leaky_20[0][0]']               
          

 conv_32 (Conv2D)               (None, None, None,   294912      ['leaky_31[0][0]']               
                                256)                                                              
                                                                                                  
 bnorm_32 (BatchNormalization)  (None, None, None,   1024        ['conv_32[0][0]']                
                                256)                                                              
                                                                                                  
 leaky_32 (LeakyReLU)           (None, None, None,   0           ['bnorm_32[0][0]']               
                                256)                                                              
                                                                                                  
 add_9 (Add)                    (None, None, None,   0           ['add_8[0][0]',                  
          

                                                                                                  
 add_12 (Add)                   (None, None, None,   0           ['add_11[0][0]',                 
                                512)                              'leaky_42[0][0]']               
                                                                                                  
 conv_44 (Conv2D)               (None, None, None,   131072      ['add_12[0][0]']                 
                                256)                                                              
                                                                                                  
 bnorm_44 (BatchNormalization)  (None, None, None,   1024        ['conv_44[0][0]']                
                                256)                                                              
                                                                                                  
 leaky_44 

                                512)                                                              
                                                                                                  
 add_16 (Add)                   (None, None, None,   0           ['add_15[0][0]',                 
                                512)                              'leaky_54[0][0]']               
                                                                                                  
 conv_56 (Conv2D)               (None, None, None,   131072      ['add_16[0][0]']                 
                                256)                                                              
                                                                                                  
 bnorm_56 (BatchNormalization)  (None, None, None,   1024        ['conv_56[0][0]']                
                                256)                                                              
          

 bnorm_66 (BatchNormalization)  (None, None, None,   2048        ['conv_66[0][0]']                
                                512)                                                              
                                                                                                  
 leaky_66 (LeakyReLU)           (None, None, None,   0           ['bnorm_66[0][0]']               
                                512)                                                              
                                                                                                  
 conv_67 (Conv2D)               (None, None, None,   4718592     ['leaky_66[0][0]']               
                                1024)                                                             
                                                                                                  
 bnorm_67 (BatchNormalization)  (None, None, None,   4096        ['conv_67[0][0]']                
          

                                                                                                  
 leaky_77 (LeakyReLU)           (None, None, None,   0           ['bnorm_77[0][0]']               
                                512)                                                              
                                                                                                  
 conv_78 (Conv2D)               (None, None, None,   4718592     ['leaky_77[0][0]']               
                                1024)                                                             
                                                                                                  
 bnorm_78 (BatchNormalization)  (None, None, None,   4096        ['conv_78[0][0]']                
                                1024)                                                             
                                                                                                  
 leaky_78 

                                128)                                                              
                                                                                                  
 bnorm_96 (BatchNormalization)  (None, None, None,   512         ['conv_96[0][0]']                
                                128)                                                              
                                                                                                  
 leaky_96 (LeakyReLU)           (None, None, None,   0           ['bnorm_96[0][0]']               
                                128)                                                              
                                                                                                  
 up_sampling2d_1 (UpSampling2D)  (None, None, None,   0          ['leaky_96[0][0]']               
                                128)                                                              
          

 leaky_104 (LeakyReLU)          (None, None, None,   0           ['bnorm_104[0][0]']              
                                256)                                                              
                                                                                                  
 conv_81 (Conv2D)               (None, None, None,   261375      ['leaky_80[0][0]']               
                                255)                                                              
                                                                                                  
 conv_93 (Conv2D)               (None, None, None,   130815      ['leaky_92[0][0]']               
                                255)                                                              
                                                                                                  
 conv_105 (Conv2D)              (None, None, None,   65535       ['leaky_104[0][0]']              
          

In [166]:
layers

{'leaky_17': <KerasTensor: shape=(None, None, None, 256) dtype=float32 (created by layer 'input_67')>}

In [108]:
model1 = keras.models.Model(inputs=model.input, outputs=model.layers[6].output)

In [136]:
model1.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv_0 (Conv2D)             (None, None, None, 32)    864       
                                                                 
 bnorm_0 (BatchNormalization  (None, None, None, 32)   128       
 )                                                               
                                                                 
 leaky_0 (LeakyReLU)         (None, None, None, 32)    0         
                                                                 
 zero_padding2d (ZeroPadding  (None, None, None, 32)   0         
 2D)                                                             
                                                                 
 conv_1 (Conv2D)             (None, None, None, 64)    184

In [182]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras import models
from keras import layers
from keras.utils.vis_utils import plot_model

resnet = model

# this is the split point, i.e. the starting layer in our sub-model
starting_layer_name = 'bnorm_0'
plot_model(model, to_file='model.png')
# create a new input layer for our sub-model we want to construct
new_input = layers.Input(batch_shape=resnet.get_layer(starting_layer_name).get_input_shape_at(0))

layer_outputs = {}
def get_output_of_layer(layer):
    # if we have already applied this layer on its input(s) tensors,
    # just return its already computed output
    if layer.name in layer_outputs:
        return layer_outputs[layer.name]

    # if this is the starting layer, then apply it on the input tensor
    if layer.name == starting_layer_name:
        out = layer(new_input)
        layer_outputs[layer.name] = out
        return out

    # find all the connected layers which this layer
    # consumes their output
    prev_layers = []
    for node in layer._inbound_nodes:
        if type(node.inbound_layers) == list:
            prev_layers.extend(node.inbound_layers)
        else:
            prev_layers.append(node.inbound_layers)
            
    print(layer.name, prev_layers, '\n')
    # get the output of connected layers
    pl_outs = []
    for pl in prev_layers:
        pl_outs.extend([get_output_of_layer(pl)])
    
    # apply this layer on the collected outputs
    out = layer(pl_outs[0] if len(pl_outs) == 1 else pl_outs)
    #print('\n',out)
    layer_outputs[layer.name] = out
    return out

# note that we start from the last layer of our desired sub-model.
# this layer could be any layer of the original model as long as it is
# reachable from the starting layer
new_output = get_output_of_layer(resnet.layers[-1])

# create the sub-model
model = models.Model(new_input, new_output)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
conv_105 [<keras.layers.advanced_activations.LeakyReLU object at 0x000001630B784220>] 

leaky_104 [<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001630B77DBB0>] 

bnorm_104 [<keras.layers.convolutional.Conv2D object at 0x000001630B778D90>] 

conv_104 [<keras.layers.advanced_activations.LeakyReLU object at 0x000001630B773E20>] 

leaky_103 [<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001630B773D60>] 

bnorm_103 [<keras.layers.convolutional.Conv2D object at 0x000001630B773730>] 

conv_103 [<keras.layers.advanced_activations.LeakyReLU object at 0x000001630B773700>] 

leaky_102 [<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001630B76FFA0>] 

bnorm_102 [<keras.layers.convolutional.Conv2D object at 0x000001630B76FBE0>] 

conv


bnorm_26 [<keras.layers.convolutional.Conv2D object at 0x000001630B6AA8E0>] 

conv_26 [<keras.layers.advanced_activations.LeakyReLU object at 0x000001630B6AA8B0>] 

leaky_25 [<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001630B6A4EE0>] 

bnorm_25 [<keras.layers.convolutional.Conv2D object at 0x000001630B6A4D30>] 

conv_25 [<keras.layers.merge.Add object at 0x000001630B6A4C40>] 

leaky_29 [<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001630B6B4760>] 

bnorm_29 [<keras.layers.convolutional.Conv2D object at 0x000001630B6AFD30>] 

conv_29 [<keras.layers.advanced_activations.LeakyReLU object at 0x000001630B6AFFD0>] 

leaky_28 [<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001630B6AFC10>] 

bnorm_28 [<keras.layers.convolutional.Conv2D object at 0x000001630B6AAFD0>] 

conv_28 [<keras.layers.merge.Add object at 0x000001630B6AF490>] 

leaky_32 [<keras.layers.normalization.batch_normali

In [167]:
middle = 60
input = keras.Input(shape=model.layers[middle-1].output_shape[1:])
# layers is a dict in the form {name : output}
layers = {}
layers[model.layers[middle-1].name] = input
for layer in model.layers[middle:]:
    if type(layer.input) == list:
        x = []
        for layer_input in layer.input:
            print(layer_input)
            x.append(layers[layer_input.name.split('/')[0]])
    else:
        x = layers[layer.input.name.split('/')[0]]
    y = layer(x)
    layers[layer.name] = y
model2 = keras.Model(inputs = [input], outputs = [y])

KerasTensor(type_spec=TensorSpec(shape=(None, None, None, 256), dtype=tf.float32, name=None), name='add_5/add:0', description="created by layer 'add_5'")


KeyError: 'add_5'

In [112]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_47 (InputLayer)          [(None, None, None,  0           []                               
                                 64)]                                                             
                                                                                                  
 leaky_1 (LeakyReLU)            (None, None, None,   0           ['input_47[0][0]']               
                                64)                                                               
                                                                                                  
 conv_2 (Conv2D)                (None, None, None,   2048        ['leaky_1[1][0]']                
                                32)                                                        

                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, None, None,   0          ['add_2[1][0]']                  
 D)                             128)                                                              
                                                                                                  
 conv_12 (Conv2D)               (None, None, None,   294912      ['zero_padding2d_2[1][0]']       
                                256)                                                              
                                                                                                  
 bnorm_12 (BatchNormalization)  (None, None, None,   1024        ['conv_12[1][0]']                
                                256)                                                              
                                                                                                  
 leaky_12 

                                128)                                                              
                                                                                                  
 conv_23 (Conv2D)               (None, None, None,   294912      ['leaky_22[1][0]']               
                                256)                                                              
                                                                                                  
 bnorm_23 (BatchNormalization)  (None, None, None,   1024        ['conv_23[1][0]']                
                                256)                                                              
                                                                                                  
 leaky_23 (LeakyReLU)           (None, None, None,   0           ['bnorm_23[1][0]']               
                                256)                                                              
          

 leaky_34 (LeakyReLU)           (None, None, None,   0           ['bnorm_34[1][0]']               
                                128)                                                              
                                                                                                  
 conv_35 (Conv2D)               (None, None, None,   294912      ['leaky_34[1][0]']               
                                256)                                                              
                                                                                                  
 bnorm_35 (BatchNormalization)  (None, None, None,   1024        ['conv_35[1][0]']                
                                256)                                                              
                                                                                                  
 leaky_35 (LeakyReLU)           (None, None, None,   0           ['bnorm_35[1][0]']               
          

                                                                                                  
 leaky_45 (LeakyReLU)           (None, None, None,   0           ['bnorm_45[1][0]']               
                                512)                                                              
                                                                                                  
 add_13 (Add)                   (None, None, None,   0           ['add_12[1][0]',                 
                                512)                              'leaky_45[1][0]']               
                                                                                                  
 conv_47 (Conv2D)               (None, None, None,   131072      ['add_13[1][0]']                 
                                256)                                                              
                                                                                                  
 bnorm_47 

                                512)                                                              
                                                                                                  
 leaky_57 (LeakyReLU)           (None, None, None,   0           ['bnorm_57[1][0]']               
                                512)                                                              
                                                                                                  
 add_17 (Add)                   (None, None, None,   0           ['add_16[1][0]',                 
                                512)                              'leaky_57[1][0]']               
                                                                                                  
 conv_59 (Conv2D)               (None, None, None,   131072      ['add_17[1][0]']                 
                                256)                                                              
          

 conv_69 (Conv2D)               (None, None, None,   524288      ['add_20[1][0]']                 
                                512)                                                              
                                                                                                  
 bnorm_69 (BatchNormalization)  (None, None, None,   2048        ['conv_69[1][0]']                
                                512)                                                              
                                                                                                  
 leaky_69 (LeakyReLU)           (None, None, None,   0           ['bnorm_69[1][0]']               
                                512)                                                              
                                                                                                  
 conv_70 (Conv2D)               (None, None, None,   4718592     ['leaky_69[1][0]']               
          

                                                                                                  
 leaky_79 (LeakyReLU)           (None, None, None,   0           ['bnorm_79[1][0]']               
                                512)                                                              
                                                                                                  
 conv_84 (Conv2D)               (None, None, None,   131072      ['leaky_79[1][0]']               
                                256)                                                              
                                                                                                  
 bnorm_84 (BatchNormalization)  (None, None, None,   1024        ['conv_84[1][0]']                
                                256)                                                              
                                                                                                  
 leaky_84 

                                128)                                                              
                                                                                                  
 leaky_99 (LeakyReLU)           (None, None, None,   0           ['bnorm_99[1][0]']               
                                128)                                                              
                                                                                                  
 conv_100 (Conv2D)              (None, None, None,   294912      ['leaky_99[1][0]']               
                                256)                                                              
                                                                                                  
 bnorm_100 (BatchNormalization)  (None, None, None,   1024       ['conv_100[1][0]']               
                                256)                                                              
          

In [114]:
model.predict(model1.predict(x))

KeyError: 'pop from an empty set'

In [128]:
# load yolov3 model and perform object detection
# based on https://github.com/experiencor/keras-yolo3
from numpy import expand_dims
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
 
# load and prepare an image
def load_image_pixels(filename, shape):
    # load the image to get its shape
    image = load_img(filename)
    width, height = image.size
    # load the image with the required size
    image = load_img(filename, target_size=shape)
    # convert to numpy array
    image = img_to_array(image)
    # scale pixel values to [0, 1]
    image = image.astype('float32')
    image /= 255.0
    # add a dimension so that we have one sample
    image = expand_dims(image, 0)
    return image, width, height
 
# load yolov3 model
# define the expected input shape for the model
input_w, input_h = 416, 416
# define our new photo
photo_filename = 'C:/Users/Jeremy/Downloads/zebra.jpg'
# load and prepare image
image, image_w, image_h = load_image_pixels(photo_filename, (input_w, input_h))
# make prediction
yhat = model.predict(image)
# summarize the shape of the list of arrays
print([a.shape for a in yhat])

[(1, 13, 13, 255), (1, 26, 26, 255), (1, 52, 52, 255)]


In [170]:
def get_layers_above(cutoff_layer,model):

  def get_next_level(layer,model):
    def wrap_list(val):
      if type(val) is list:
        return val
      return [val] 
    r=[]
    for output_t in wrap_list(layer.output):
      r+=[x for x in model.layers if output_t.name in [y.name for y in wrap_list(x.input)]]
    return r

  visited=set()
  to_visit=set([cutoff_layer])

  while to_visit:
    layer=to_visit.pop()
    to_visit.update(get_next_level(layer,model))
    visited.add(layer)
  return list(visited)

In [181]:
layers=get_layers_above(model.get_layer('conv_0'),model)
print([l.name for l in layers])

['conv_104', 'conv_77', 'zero_padding2d_1', 'leaky_3', 'bnorm_67', 'conv_12', 'leaky_92', 'leaky_59', 'leaky_104', 'conv_10', 'add_11', 'add_13', 'add_15', 'bnorm_1', 'bnorm_10', 'leaky_9', 'leaky_13', 'leaky_89', 'leaky_32', 'add_9', 'conv_3', 'leaky_99', 'conv_100', 'zero_padding2d', 'conv_34', 'bnorm_91', 'leaky_96', 'leaky_75', 'up_sampling2d_1', 'conv_76', 'bnorm_60', 'bnorm_37', 'leaky_38', 'conv_39', 'bnorm_23', 'leaky_44', 'conv_45', 'bnorm_29', 'leaky_50', 'conv_51', 'bnorm_12', 'bnorm_35', 'leaky_56', 'conv_0', 'conv_57', 'conv_54', 'leaky_19', 'bnorm_0', 'conv_20', 'leaky_25', 'leaky_53', 'conv_92', 'conv_26', 'leaky_88', 'conv_89', 'leaky_73', 'conv_14', 'concatenate_1', 'add_22', 'bnorm_41', 'leaky_62', 'leaky_42', 'conv_63', 'bnorm_47', 'leaky_48', 'conv_93', 'conv_99', 'add_12', 'bnorm_90', 'bnorm_53', 'conv_75', 'leaky_91', 'add_14', 'conv_96', 'leaky_64', 'leaky_60', 'leaky_37', 'conv_62', 'conv_38', 'conv_44', 'leaky_23', 'bnorm_100', 'add_18', 'conv_50', 'leaky_29', 